# Parsing SEC 13F forms

We examine 13F filings which are quarterly reports filed per SEC regulations
by institutional investment managers containing all equity assets under management
of at least \$ 100 million in value. **Form 13F is required to be filed
within 45 days of the end of a calendar quarter**
(*which should be considered as significant information latency*).

Form 13F *only reports long* positions.
Short positions are not required to be disclosed and are not reported.
Section 13(f) securities generally include equity securities
that trade on an exchange (including Nasdaq), certain equity options and warrants,
shares of closed-end investment companies, and certain convertible debt securities.
The shares of open-end investment companies
(i.e. mutual funds) are not Section 13(f) securities.
See [Official List of Section 13(f) Securities](http://www.sec.gov/divisions/investment/13flists.htm) and our caveats section below.

Form 13F surprisingly excludes total portfolio value and percentage allocation
of each stock listed. We remedy that, and also parse the report for easy reading.
Our notebook then develops into a module **yi_secform** which will do
all the work via one function.

As specific example, we follow Druckenmiller and Paulson as asset managers
who have significant positions in GLD, a gold ETF.
We show the Druckenmiller's sudden accumulation,
and Paulson's dramatic liquidation.

*Top holdings are easily analyzed by a single Python module:* **yi_secform**
Caveats are disclosed in the first section.

Shortcut to this notebook: https://git.io/13F

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235 -- Module: yi_secform
- Python: pandas, numpy, lxml, bs4, html5lib
     
*CHANGE LOG*

    2016-02-22  Fix issue #2 by v4 and p6 updates.
                   Replace .sort(columns=...) with .sort_values(by=...)
                   since pandas 0.17.1 gives us future deprecation warning.
                   Paulson liquidates 37.6% of his GLD inventory.
    2015-11-16  Update in Appendix for Druckenmiller and Paulson.
    2015-08-28  First version.

In [1]:
from fecon235.fecon235 import *

#  pandas will give best results
#  if it can call the Python package: lxml,
#  and as a fallback: bs4 and html5lib.
#  They parse (non-strict) XML and HTML pages.
#  Be sure those three packages are pre-installed.

from fecon235.lib import yi_secform
#  We are going to derive this module in this notebook.

In [2]:
#  PREAMBLE-p6.15.1223 :: Settings and system details
from __future__ import absolute_import, print_function
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
#  Beware, for MATH display, use %%latex, NOT the following:
#                   from IPython.display import Math
#                   from IPython.display import Latex
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.11
 ::  IPython 4.0.0
 ::  jupyter 1.0.0
 ::  notebook 4.0.6
 ::  matplotlib 1.4.3
 ::  numpy 1.10.1
 ::  pandas 0.17.1
 ::  pandas_datareader 0.2.0
 ::  Repository: fecon235 v4.16.0221 develop
 ::  Timestamp: 2016-02-23, 19:11:16 UTC
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


## Caveats regarding 13F

- 13F filings disclose hedge fund long positions in US equity markets, American Depositary Receipts (ADRs), both put and call options, as well as convertible notes. **They do not disclose short sales, cash positions, or any other asset class.**

- Yet performance results for the [clones](http://blog.alphaclone.com/alphaclone/2011/01/clone-vs-fund-2010.html) are somewhat in line with the actual performance of the fund. E.g. Paulson & Co (John Paulson)

- Clones track managers that normally run net long. Tracking global macro funds (Bridgewater, Tudor) or credit funds (Fortress, Cerberus) is misguided because the vast majority of their positions are in asset classes that they don't have to disclose (futures, commodities, bonds, currencies, etc).  And while quant funds (RenTec, AQR) often disclose stocks, following them is a folly because you have absolutely no idea why their algorithms bought in the first place (statistical arbitrage may entail positions in foreign countries).  

- 13F does not reveal international holdings (except for ADR's).

- Follow long-term oriented funds to reduce the effect of the delay in 13F disclosures.

- Money managers allocate the most capital to their best ideas. Pay attention to "new positions" in their disclosures as these are their most recent ideas. 

- Always remember that the **13F is not their whole portfolio and that it's a past snapshot.**

- Monitor all SEC filings, not just 13F's:  13G filings, 13D filings, as well as various Form 3 and Form 4's are filed on a more timely basis and provide a more current look at what managers are buying or selling.  They are required to file when they've purchased 5% or more of a company. 

- Caveats source: http://www.marketfolly.com/2012/10/hedge-fund-13f-filing-pros-and-cons.html

## Analysis of the 13F format

Source: http://www.sec.gov/answers/form13f.htm


### Obtaining 13F

You can search for and retrieve Form 13F filings using the [SEC's EDGAR database](http://www.sec.gov/edgar/searchedgar/companysearch.html). To find the filings of a particular money manager, enter the money manager's name in the Company Name field. To see all recently filed 13Fs, use the ["Latest Filings"](http://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent) search function and enter "13F" in the Form Type box.


### Example: Druckenmiller, 14 August 2015 filing

Stanley Druckenmiller closed his Duquesne Capital Management hedge fund in 2010 but he still discloses the holdings of his family office (search: [“Duquesne Family Office”](http://www.sec.gov/cgi-bin/browse-edgar?company=Duquesne+Family+Office&owner=exclude&action=getcompany) picking the Information Table in html).

He made headlines because of his new acquisition in gold (the GLD ETF) which now is his largest position (about 22% of his entire portfolio)!

In [3]:
#            https cannot be read by lxml, surprisingly.
druck150814='http://www.sec.gov/Archives/edgar/data/1536411/000153641115000006/xslForm13F_X01/form13f_20150630.xml'

## HOWTO parse 13F reports: deriving our module

If bugs appear because of format changes at the SEC
this section will useful for interactive debugging.

Take note that: **lxml** *cannot read https, so use http
when specifying URL.*

In [4]:
#     START HERE with a particular URL:
url = druck150814

In [5]:
#  Let's display the web page as in the browser to understand the semantics:
HTML("<iframe src=" + url + " width=1400 height=350></iframe>")

In [6]:
#  Use pandas to read in the xml page...
#  See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html

#  It searches for <table> elements and only for <tr> and <th> rows and <td> elements 
#  within each <tr> or <th> element in the table.

page = pd.read_html( url )

In [7]:
#  Nasty output in full:

#uncomment:  page

In [8]:
#  page is a list of length 4:
len( page )

4

In [9]:
#  But only the last element of page interests us:
df = page[-1]
#  which turns out to be a dataframe!

In [10]:
#  Let's rename columns for our sanity:
df.columns = [ 'stock', 'class', 'cusip', 'usd', 'size', 'sh_prin', 'putcall', 'discret', 'manager', 'vote1', 'vote2', 'vote3'] 

In [11]:
#  But first three rows are SEC labels, not data, 
#  so delete them:
df = df[3:]

#  Start a new index from 0 instead of 3:
df.reset_index( drop=True )

                            stock           class      cusip     usd     size  \
0                 Akamai Tech Inc             COM  00971T101   24807   355300   
1                       Alcoa Inc             COM  013817101   40698  3650000   
2          Alibaba Group Hldg Ltd       SPONS ADS  01609W102   18025   219100   
3                Am Int'l Grp Inc             COM  026874784   15474   250300   
4                       BB&T Corp             COM  054937107   19575   485600   
5                   Citigroup Inc             COM  172967424   64907  1175000   
6       Cognizant Tech Sol's Corp            CL A  192446102   14466   236800   
7             Ctrip.com Int'l Ltd      AM DEP SHS  22943F100   14626   201400   
8                 Dow Chemical Co             COM  260543103   59526  1163300   
9                   EOG Res's Inc             COM  26875P101   27219   310900   
10            Eastman Chemical Co             COM  277432100   15219   186000   
11                       EBa

In [12]:
#  Delete irrevelant columns:
dflite = df.drop( df.columns[[1, 4, 5, 7, 8, 9, 10, 11]], axis=1 )
#         inplac=True only after pandas 0.13
#uncomment: dflite

In [13]:
#  usd needs float type since usd was read as string:
dflite[['usd']] = dflite[['usd']].astype( float )
#                  Gotcha: int as type will fail for NaN

#  Type change allows proper sort:
dfusd = dflite.sort_values( by=['usd'], ascending=[False] )

In [14]:
usdsum = sum( dfusd.usd )
#  Portfolio total in USD:
usdsum

1484044.0

In [15]:
#  New column for percentage of total portfolio:
dfusd['pcent'] = np.round(( dfusd.usd / usdsum ) * 100, 2)

In [16]:
#  Top 20 Hits!
dfusd.head( 20 )

                      stock      cusip     usd putcall  pcent
27          SPDR Gold Trust  78463V907  323626     NaN  21.81
15             Facebook Inc  30303M102  160612     NaN  10.82
29         Wells Fargo & Co  949746101   94449     NaN   6.36
31  LyondellBasell Ind's NV  N53745100   74219     NaN   5.00
18           Halliburton Co  406216101   66629     NaN   4.49
16     Freeport-McMoRan Inc  35671D857   66045     NaN   4.45
8             Citigroup Inc  172967424   64907     NaN   4.37
22           Microsoft Corp  594918104   61726     NaN   4.16
17            HDFC Bank Ltd  40415F101   60748     NaN   4.09
11          Dow Chemical Co  260543103   59526     NaN   4.01
20     JP Morgan Chase & Co  46625H100   52670     NaN   3.55
19             Illumina Inc  452327109   45091     NaN   3.04
4                 Alcoa Inc  013817101   40698     NaN   2.74
23      Newmont Mining Corp  651639106   29898     NaN   2.01
12            EOG Res's Inc  26875P101   27219     NaN   1.83
30      

## Summary module: yi_secform

**We sum up our work above in a module for easy execution using one function.**

In [17]:
yi_secform.pcent13f??

Signature: yi_secform.pcent13f(url='http://www.sec.gov/Archives/edgar/data/1536411/000153641115000006/xslForm13F_X01/form13f_20150630.xml', top=7654321)
Source:
def pcent13f( url=druck150814, top=7654321 ):
     '''Prune, then sort SEC 13F by percentage allocation, showing top N.
     >>> pcent13f( top= 7 )
                           stock      cusip     usd putcall  pcent
     27          SPDR Gold Trust  78463V907  323626     NaN  21.81
     15             Facebook Inc  30303M102  160612     NaN  10.82
     29         Wells Fargo & Co  949746101   94449     NaN   6.36
     31  LyondellBasell Ind's NV  N53745100   74219     NaN   5.00
     18           Halliburton Co  406216101   66629     NaN   4.49
     16     Freeport-McMoRan Inc  35671D857   66045     NaN   4.45
     8             Citigroup Inc  172967424   64907     NaN   4.37
     '''
     df = parse13f( url )
     #    Drop irrevelant COLUMNS:
     df.drop( df.columns[[1, 4, 5, 7, 8, 9, 10, 11]], axis=1, inplace=True )
     #  

In [18]:
yi_secform.pcent13f( druck150814, 20 )

#  Simply enter the Information Table html URL for a 13F filing, 
#  and bang... [verifying our output in the previous cell]:

                      stock      cusip     usd putcall  pcent
27          SPDR Gold Trust  78463V907  323626     NaN  21.81
15             Facebook Inc  30303M102  160612     NaN  10.82
29         Wells Fargo & Co  949746101   94449     NaN   6.36
31  LyondellBasell Ind's NV  N53745100   74219     NaN   5.00
18           Halliburton Co  406216101   66629     NaN   4.49
16     Freeport-McMoRan Inc  35671D857   66045     NaN   4.45
8             Citigroup Inc  172967424   64907     NaN   4.37
22           Microsoft Corp  594918104   61726     NaN   4.16
17            HDFC Bank Ltd  40415F101   60748     NaN   4.09
11          Dow Chemical Co  260543103   59526     NaN   4.01
20     JP Morgan Chase & Co  46625H100   52670     NaN   3.55
19             Illumina Inc  452327109   45091     NaN   3.04
4                 Alcoa Inc  013817101   40698     NaN   2.74
23      Newmont Mining Corp  651639106   29898     NaN   2.01
12            EOG Res's Inc  26875P101   27219     NaN   1.83
30      

## Quick look at John Paulson

For comparison, since the press has been stating Paulson has been selling the GLD ETF due to redemptions. 13F search page: http://www.sec.gov/cgi-bin/browse-edgar?company=Paulson+%26+Co.&owner=exclude&action=getcompany

In [19]:
#  13F for Paulson & Co. filed 2015-08-14:
paulson150814 = 'http://www.sec.gov/Archives/edgar/data/1035674/000114036115032242/xslForm13F_X01/form13fInfoTable.xml'

In [20]:
yi_secform.pcent13f( paulson150814, 20 )

                           stock      cusip      usd putcall  pcent
4                   ALLERGAN PLC  G0177J108  2177083     NaN  10.04
59  VALEANT PHARMACEUTICALS INTL  91911K102  1999350     NaN   9.22
49                     SHIRE PLC  82481R106  1713396     NaN   7.90
36                     MYLAN N V  N59465109  1487020     NaN   6.86
56         TIME WARNER CABLE INC  88732J207  1447275     NaN   6.67
19                       DIRECTV  25490A309  1057806     NaN   4.88
50               SPDR GOLD TRUST  78463V107  1037720     NaN   4.79
52  STARWOOD HOTELS&RESORTS WRLD  85590A401   973031     NaN   4.49
58               T-MOBILE US INC  872590104   924467     NaN   4.26
5        AMERICAN INTL GROUP INC  026874784   902770     NaN   4.16
21        EXTENDED STAY AMER INC  30224P200   894608     NaN   4.13
32       MALLINCKRODT PUB LTD CO  G5785G107   776268     NaN   3.58
23                   GRIFOLS S A  398438408   551918     NaN   2.55
25              HCA HOLDINGS INC  40412C101   42

### Gold ETF comment, August 2015

Paulson is usually the largest stakeholder in GLD also known as "SPDR GOLD TRUST". The latest 13F shows he holds \$1.037 billion worth which is equivalent to about 886,183 troy ounces (*27.6 metric tons*). It actually only represents half of his very top equity holding: Allergan at 10% allocation.

Druckenmiller's 13F disclosing his Family Office (not a public operation like Paulson & Co) shows a \$0.324 billion position in GLD, equivalent to about 276,367 troy ounces (*8.6 metric tons*). It appears to be a new bold trade since it is his largest holding at 22% allocation. 

[Spot gold on 2015-06-30, end of second calendar quarter, was fixed at \$1171 in London.]

*We do not truly know their respective net positions because they **could be hedged in the gold futures market.** Futures positions are excluded from 13F filings, as well as cash position (which would fund any further accumulation).*

Remember that we viewing past snapshots of positions dated 30 June 2015. Such positions could have been entirely liquidated during July and August when the gold market declined severely. On the other hand, gold rallied considerably days after the market received Druckenmiller's vote of bold confidence via his 13F filed on August 14th. 

## Appendix: Getting quotes

GLD, the ETF for gold, is designed to track spot gold prices,
less their management fees.
Within fecon235, one can easily retrieve stock or ETF quotes,
for example, by this syntax: get('s4gld'),
noting the string 's4' concatenated with the symbol
in lower case.

Notice that the SEC 13F requires unique CUSIP identifiers,
rather than ticker symbols for their 13F forms.

For gold, we prefer to use equivalent measures used in the spot market.
For example, quotes here are given in USD per troy ounce,
and the London PM fix (rather than nearby futures) is
accepted as a benchmark.
GLD valuation can be converted into such spot terms.
Within fecon235: get(d4xau)
will retrieve the appropriate dataframe for you.
See https://git.io/gold for more details.

## Appendix: November 2015 update

In [21]:
druck151113 = 'http://www.sec.gov/Archives/edgar/data/1536411/000153641115000008/xslForm13F_X01/form13f_20150930.xml'
paulson151116 = 'http://www.sec.gov/Archives/edgar/data/1035674/000114036115041689/xslForm13F_X01/form13fInfoTable.xml'

In [22]:
# Druckenmiller 13F for 2015-11-13:
yi_secform.pcent13f( druck151113, 20 )

                        stock      cusip     usd putcall  pcent
9                Facebook Inc  30303M102  367988     NaN  31.29
17            SPDR Gold Trust  78463V907  307757    Call  26.17
13             Microsoft Corp  594918104   61477     NaN   5.23
10              HDFC Bank Ltd  40415F101   61310     NaN   5.21
19           Wells Fargo & Co  949746101   61050     NaN   5.19
3              Amazon.com Inc  023135106   49807     NaN   4.23
5         Ctrip.com Int'l Ltd  22943F100   30364     NaN   2.58
20                Workday Inc  98138H101   30187     NaN   2.57
12                 JD.Com Inc  47215P106   28103     NaN   2.39
11               Illumina Inc  452327109   27393     NaN   2.33
18          Under Armour, Inc  904311107   25143     NaN   2.14
8                 Expedia Inc  30212P303   20370     NaN   1.73
4               Carnival Corp  143658300   19303     NaN   1.64
16        Priceline Group Inc  741503403   19295     NaN   1.64
21  Royal Carib'n Cruises Ltd  V7780T103

In [23]:
#  Paulson 13F for 2015-11-16:
yi_secform.pcent13f( paulson151116, 20 )

                           stock      cusip      usd putcall  pcent
5                   ALLERGAN PLC  G0177J108  1952139     NaN  10.13
63  VALEANT PHARMACEUTICALS INTL  91911K102  1585798     NaN   8.23
61         TIME WARNER CABLE INC  88732J207  1457023     NaN   7.56
54                     SHIRE PLC  82481R106  1396631     NaN   7.25
56  STARWOOD HOTELS&RESORTS WRLD  85590A401  1036756     NaN   5.38
59  TEVA PHARMACEUTICAL INDS LTD  881624209  1016280     NaN   5.28
55               SPDR GOLD TRUST  78463V107   986836     NaN   5.12
62               T-MOBILE US INC  872590104   937430     NaN   4.87
38                     MYLAN N V  N59465109   882220     NaN   4.58
7        AMERICAN INTL GROUP INC  026874784   829754     NaN   4.31
24        EXTENDED STAY AMER INC  30224P200   799761     NaN   4.15
26                   GRIFOLS S A  398438408   538966     NaN   2.80
45                PERRIGO CO PLC  G97822103   462138     NaN   2.40
34       MALLINCKRODT PUB LTD CO  G5785G107   42

[Spot gold on 2015-09-30, end of third calendar quarter, was fixed 
at $1114 in London, -4.9% from previous quarter.]

2015-11-16: GLD is no longer Druckenmiller's top holding (replaced by Facebook): 
change from last quarter "323626 NaN 21.81" to "307757 Call 26.17" -- 
however, its allocation has been increased, though net position is 
unchanged (given price decrease in the spot market). 
Curiously it appears the underlying instrument has *shifted to calls*. 
The expiration date of the calls are not known, but it is 
indicative of a shift to short-term trading perspective. 
Druckenmiller's gold downside is now limited to the premiums paid.

As for Paulson: change from last quarter: "1037720 NaN 4.79" to "986836 NaN 5.12" -- 
indicates *no change in position* since the GLD valuation 
mirrors the decrease in the spot gold market.

## Appendix: February 2016 update

In [24]:
druck160216='http://www.sec.gov/Archives/edgar/data/1536411/000153641116000010/xslForm13F_X01/form13f_20151231.xml'
paulson160216='http://www.sec.gov/Archives/edgar/data/1035674/000114036116053318/xslForm13F_X01/form13fInfoTable.xml'

In [25]:
# Druckenmiller 13F for 2016-02-16:
yi_secform.pcent13f( druck160216, 20 )

                      stock      cusip     usd putcall  pcent
13          SPDR Gold Trust  78463V907  292205    Call  29.90
7             Facebook Inc.  30303M102  264842     NaN  27.10
4           Amazon.com Inc.  023135106  128419     NaN  13.14
9            Microsoft Corp  594918104   64634     NaN   6.61
12              Raytheon Co  755111507   52029     NaN   5.32
6      Ctrip.com Int'l Ltd.  22943F100   35781     NaN   3.66
8            HDFC Bank Ltd.  40415F101   30252     NaN   3.10
10    Northrop Grumman Corp  666807102   25603     NaN   2.62
3             Alphabet Inc.  02079K305   25519     NaN   2.61
11        Pure Storage Inc.  74624M102   24912     NaN   2.55
14           Syncrony Fin'l  87165B103   14402     NaN   1.47
5   Chipotle Mex Grill Inc.  169656105    9693     NaN   0.99
15  Tempur Sealy Int'l Inc.  88023U101    8843     NaN   0.90

In [26]:
#  Paulson 13F for 2016-02-16:
yi_secform.pcent13f( paulson160216, 20 )

                           stock      cusip      usd putcall  pcent
6                   ALLERGAN PLC  G0177J108  1728938     NaN  10.33
65                     SHIRE PLC  82481R106  1403184     NaN   8.39
75  VALEANT PHARMACEUTICALS INTL  91911K102  1348479     NaN   8.06
71  TEVA PHARMACEUTICAL INDS LTD  881624209  1339666     NaN   8.01
47                     MYLAN N V  N59465109  1232475     NaN   7.37
73         TIME WARNER CABLE INC  88732J207  1089246     NaN   6.51
67  STARWOOD HOTELS&RESORTS WRLD  85590A401   803882     NaN   4.81
74               T-MOBILE US INC  872590104   773747     NaN   4.63
8        AMERICAN INTL GROUP INC  026874784   719050     NaN   4.30
29        EXTENDED STAY AMER INC  30224P200   678319     NaN   4.05
42       MALLINCKRODT PUB LTD CO  G5785G107   643586     NaN   3.85
66               SPDR GOLD TRUST  78463V107   585933     NaN   3.50
32                   GRIFOLS S A  398438408   401700     NaN   2.40
38             LAM RESEARCH CORP  512807108   39

[Spot gold on 2015-12-31, end of fourth calendar quarter, was fixed at $1060 in London, -4.85% from previous quarter.]

2016-02-21: GLD is back to Druckenmiller's top holding: change from last quarter "307757 Call 26.17" to "292205 Call 29.90". If the net position was unchanged (given price decrease in the spot market), we can attribute a loss of \$ 626,000 due to decay in the call option valuation (cf. theta).

#### 2015-Q4: Drama for Paulson

As for Paulson: change from last quarter: "986836 NaN 5.12" to "585933 NaN 3.50" -- indicates HUGE change! Market movement alone would give 938974 valuation, so the
**realized displacement on GLD is \$ 353,041,000, i.e. -353 million USD.** 

Paulson is usually the largest stakeholder in GLD also known as "SPDR GOLD TRUST".
The 13F parsed in August 2015 showed \$ 1.037 billion worth which was equivalent to
about 886,183 troy ounces (27.6 metric tons).
The 13F currently shows \$ 0.586 billion worth which is equivalent to
about 552,767 troy ounces -- a reduction of -37.6% in gold inventory.

Thus we can estimate that *Paulson liquidated the equivalent of
about 333,416 troy ounces at a weighted average price of \$ 1059*
(note: recorded low during 2016-Q4 for the London PM Gold fix was \$ 1049).
From that weighted average and gold's price history,
we can discern that the liquidation took place largely in late December 2015.